In [1]:
import pandas as pd

In [2]:
df_clean = pd.read_csv("imputation_outlier_encoding_alldone.csv")  # Load your dataset

In [3]:
# Convert Quote Date to datetime
df_clean['Quote Date'] = pd.to_datetime(df_clean['Quote Date'])

# Step 1: Profile Complexity based on Profile Name frequency rank
# Rank and bucket into 5 complexity levels (1–5)
profile_ranks = pd.qcut(df_clean['Profile Name'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5])
df_clean['Profile_Complexity'] = profile_ranks.astype(int)

# Step 2: Manufacturing Difficulty = Tolerances * (GD&T + 1)
df_clean['Manufacturing_Difficulty'] = df_clean['Tolerances'] * (df_clean['GD&T'] + 1)

# Step 3: Time-Series Features for LME Price
df_clean = df_clean.sort_values('Quote Date')
df_clean['LME_MA_7'] = df_clean['LME price'].rolling(window=7, min_periods=1).mean()
df_clean['LME_Lag_1'] = df_clean['LME price'].shift(1).bfill()

#import ace_tools as tools; tools.display_dataframe_to_user(name="Feature Engineered Final Dataset", dataframe=df_clean)


In [4]:
df_clean.head(5)

,Alloy,Finish,Length,Weight,Tolerances,GD&T,Order Quantity,LME price,Customer Category,Lead Time (weeks),Quote Price (SEK),Quote Date,Profile Name,Profile_Complexity,Manufacturing_Difficulty,LME_MA_7,LME_Lag_1
821,0.0,4.0,17.6,1.718,0.147,0.0,50862,2.75,2.0,11.0,3.83,2025-01-01,2.993724,1,0.147,2.750000,2.75
599,1.0,3.0,27.0,1.245,0.067,2.0,116471,2.96,1.0,9.0,2.67,2025-01-01,3.096523,5,0.201,2.855000,2.75
489,2.0,4.0,24.1,0.885,0.099,2.0,63929,3.97,2.0,7.0,3.60,2025-01-02,3.041944,4,0.297,3.226667,2.96
750,2.0,4.0,26.1,1.042,0.152,1.0,120204,2.83,3.0,10.0,2.61,2025-01-02,3.146656,5,0.304,3.127500,3.97
924,2.0,3.0,34.8,1.368,0.118,0.0,94419,3.59,1.0,2.0,3.06,2025-01-03,3.014675,3,0.118,3.220000,2.83


In [5]:
output_file = "final.csv"
df_clean.to_csv(output_file, index=False)